In [1]:
import pandas as pd
import numpy as np
from random import normalvariate
import plotly.express as px

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

In [2]:
initial_state = {
    'co2': 400,             # 0.04% = 400 ppm
    'temperature': 290      # T-273.15 = 16.85 degrees celcius
}

In [3]:
system_params = {
    'sun_radiation': [1361], # solar constant, the average amount of solar radiation received at the outer atmosphere of the Earth on a unit area perpendicular to the Sun's rays in W/m²
    'temperature_constant': [1e-4],
    'co2_reflectance_factor': [1e-3],
    'co2_gigatons_to_ppm': [1.2e-1],
    'co2_stdev': [40],
    'heat_dissipation_constant': [2075],
    'co2_annual_emissions': [40, 80, 40, 80, 40, 80, 40, 80],
    'year_of_the_wakening': [0, 0, 10, 10, 50, 50, 100, 100]
}

In [4]:
def p_co2_emissions(params, substep, state_history, previous_state):
    # Parameters & variables
    mean = params['co2_annual_emissions']
    std = params['co2_stdev']
    conversion_factor = params['co2_gigatons_to_ppm']
    t_w = params['year_of_the_wakening']
    t = previous_state['timestep']
    
    # Logic
    if t > t_w:
        mean = 0
    
    value = normalvariate(mean, std) * conversion_factor
    
    # Output
    return {'add_co2': value}

In [5]:
def p_sun_radiation(params, substep, state_history, previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    a = params['sun_radiation']
    
    # Logic
    temp_change = g * a
    
    # Output
    return {'add_temperature': temp_change}

In [6]:
def p_earth_cooling(params, substep, state_history, previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    K = params['heat_dissipation_constant']
    T = previous_state['temperature']
    
    temp_change = -(g * K * (T / 300) ** 4)
    
    # Output
    return {'add_temperature': temp_change}

In [7]:
def p_greenhouse_effect(params, substep, state_history, previous_state):
    g = params['temperature_constant']
    K = params['heat_dissipation_constant']
    beta = params['co2_reflectance_factor']
    T = previous_state['temperature']
    CO2 = previous_state['co2']
    
    alpha = 1 - np.exp(-beta * CO2)
    temp_change = g * alpha * K * (T / 300) ** 4
    
    return {'add_temperature': temp_change}

In [8]:
def s_co2(params, substep, state_history, previous_state, policy_input):
    current_co2 = previous_state['co2']
    co2_change = policy_input['add_co2']
    
    new_co2 = max(current_co2 + co2_change, 0)
    
    return 'co2', new_co2

In [9]:
def s_temperature(params, substep, state_history, previous_state, policy_input):
    # Parameters & variables
    current_temp = previous_state['temperature']
    temp_change = policy_input['add_temperature']
    
    new_temp = max(current_temp + temp_change, 0)
    
    return 'temperature', new_temp

In [10]:
partial_state_update_blocks = [
    # ran in series
    {
        'label': 'Temperature dynamics',
        'policies': {
            'sun_ratiation': p_sun_radiation,
            'earth_cooling': p_earth_cooling,
            'greenhouse_effect': p_greenhouse_effect
        },
        'variables': {
            'temperature': s_temperature
        }
    },
    
    {
        'label': 'CO2 dynamics',
        'policies': {
            'co2_emissions': p_co2_emissions,
        },
        'variables': {
            'co2': s_co2
        }
    }
]

In [11]:
MONTE_CARLO_RUNS = 50
SIMULATION_TIMESTEPS = 100

sim_config = config_sim(
    {
        'N': MONTE_CARLO_RUNS,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params, # parameters to sweep over
    }
)

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    # this generates a list of several configuration objects. The states number of monte carlo runs are performed for
    # combinations of swept parameters
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks
)

In [12]:
# The ExecutionContext decides how cadCAD should run the simulation by selecting an execution mode. The default selects
# between single-threaded and multi-process threaded modes. This optimizes the performance of the simulation or how fast
# it runs.

exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 8
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100, 8, 50, 2)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4

In [13]:
df = (pd.DataFrame(system_events)
      .assign(years = lambda df: df.timestep)
      .assign(temperature_celcius = lambda df: df.temperature - 273)
      .query('timestep > 0')
)

# Clean substeps
# Aside from each point in time (timesteps), cadCAD generates points for each intermediate execution through the PSUBs.
# These points are known as substeps. If 3 blocks, then 3 substeps per timestep.

first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])

# Attribute parameters to each row
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])
    
df

,co2,temperature,simulation,subset,run,timestep,years,temperature_celcius,sun_radiation,temperature_constant,co2_reflectance_factor,co2_gigatons_to_ppm,co2_stdev,heat_dissipation_constant,co2_annual_emissions,year_of_the_wakening
2,394.635582,290.014647,0,0,1,1,1,17.014647,1361,0.0001,0.001,0.12,40,2075,40,0
4,398.992850,290.028617,0,0,1,2,2,17.028617,1361,0.0001,0.001,0.12,40,2075,40,0
6,395.290043,290.043093,0,0,1,3,3,17.043093,1361,0.0001,0.001,0.12,40,2075,40,0
8,405.983377,290.057095,0,0,1,4,4,17.057095,1361,0.0001,0.001,0.12,40,2075,40,0
10,411.863684,290.072372,0,0,1,5,5,17.072372,1361,0.0001,0.001,0.12,40,2075,40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80391,1317.709674,295.231354,7,7,50,96,96,22.231354,1361,0.0001,0.001,0.12,40,2075,80,100
80393,1328.702605,295.315345,7,7,50,97,97,22.315345,1361,0.0001,0.001,0.12,40,2075,80,100
80395,1341.549380,295.399847,7,7,50,98,98,22.399847,1361,0.0001,0.001,0.12,40,2075,80,100
80397,1350.492028,295.484950,7,7,50,99,99,22.484950,1361,0.0001,0.001,0.12,40,2075,80,100
